In [52]:
import pandas as pd
import numpy as np
import scipy.stats as stats
#regression
import statsmodels.api as sm
from linearmodels import PanelOLS as p

In [53]:
base=pd.read_stata('Star13.dta')

In [130]:
base=base.fillna(0)

#Dummy variable for free lunch

base['freelunchk']=np.where(base['sesk']==1,1,0)
base['freelunch1']=np.where(base['ses1']==1,1,0)
base['freelunch2']=np.where(base['ses2']==1,1,0)
base['freelunch3']=np.where(base['ses3']==1,1,0)

#Dummy variable for k attrition

base['attritionk']=np.where(base['stark']==2,1,0)
base['attritionk']=np.where(base['star1']==2,1,base['attritionk'])
base['attritionk']=np.where(base['star2']==2,1,base['attritionk'])
base['attritionk']=np.where(base['star3']==2,1,base['attritionk'])

#Dummy variable for 1 attrition
base['attrition1']=np.where(base['star1']==2,1,0)
base['attrition1']=np.where(base['star2']==2,1,base['attrition1'])
base['attrition1']=np.where(base['star3']==2,1,base['attrition1'])

#Dummy Variable for 2 attrition

base['attrition2']=np.where(base['star2']==2,1,0)
base['attrition2']=np.where(base['star3']==2,1,base['attrition2'])


#Dummy variable for white/asian
base['wa']=np.where(base['race']==1,1,0)
base['wa']=np.where(base['race']==3,1,base['wa'])

#Age in 1985
base['age']=1985-base['yob']
base=base[base['age']>0]

#avg percentile score K

base['score']=(base['mathk_p']+base['readk_p'])/2
base['score']=base['score'].fillna(0)
base['score']=np.where(base['score']<2, (base['math1_p']+base['read1_p'])/2, base['score'])
base['score']=base['score'].fillna(0)
base['score']=np.where(base['score']<2, (base['math2_p']+base['read2_p'])/2, base['score'])
base['score']=base['score'].fillna(0)
base['score']=np.where(base['score']<2, (base['math3_p']+base['read3_p'])/2, base['score'])

#avg percentile score 1st

base['score1']=(base['math1_p']+base['read1_p'])/2
base['score1']=base['score1'].fillna(0)
base['score1']=np.where(base['score1']==0, (base['math2_p']+base['read2_p'])/2, base['score1'])
base['score1']=base['score1'].fillna(0)
base['score1']=np.where(base['score1']==0, (base['math3_p']+base['read3_p'])/2, base['score1'])
base['score1']=base['score1'].fillna(0)
base['score1']=np.where(base['score1']==0, (base['mathk_p']+base['readk_p'])/2, base['score1'])

#avg percentile score 2nd

base['score2']=(base['math2_p']+base['read2_p'])/2
base['score2']=base['score2'].fillna(0)
base['score2']=np.where(base['score2']==0, (base['math3_p']+base['read3_p'])/2, base['score2'])

#avg percentile score 3rd

base['score3']=(base['math3_p']+base['read3_p'])/2


#remove top coded schoolid
base['schidk']=np.where(base['schidk']==999,'',base['schidk'])

#add a dummy for gender

base['sex']=np.where(base['sex']==2,1,0)

#small class dummy

base['smallc']=np.where(base['ctypek']==1,1,0)
base['regAc']=np.where(base['ctypek']==3,1,0)
#base['regAc']=np.where(base['csizek']>16,1,base['regAc'])
base['reg']=np.where(base['ctypek']==2,1,0)

#student counter for F-test

base['student']=1

#scores for regressions

base['kscore']=(base['mathk_p']+base['readk_p'])/2

#print (base[['score', 'mathk_p', 'readk_p']])

print (base.columns.values)


['id' 'sex' 'race' 'mathk' 'readk' 'math1' 'read1' 'math2' 'read2' 'math3'
 'read3' 'stark' 'ctypek' 'star1' 'ctype1' 'star2' 'ctype2' 'star3'
 'ctype3' 'csizek' 'csize1' 'csize2' 'csize3' 'sesk' 'ses1' 'ses2' 'ses3'
 'yob' 'schidk' 'schid1' 'schid2' 'schid3' 'mathk_p' 'math1_p' 'math2_p'
 'math3_p' 'readk_p' 'read1_p' 'read2_p' 'read3_p' 'freelunchk'
 'freelunch1' 'freelunch2' 'freelunch3' 'attritionk' 'attrition1'
 'attrition2' 'wa' 'age' 'score' 'score1' 'smallc' 'regAc' 'reg' 'student'
 'kscore' 'init' 'score2' 'score3']


In [55]:
# Calculating f-statistics.  Something isn't right here.
base.head()
y=base[['schidk', 'ctypek', 'freelunchk', 'student']]
y=y.groupby(['schidk', 'ctypek']).sum()
y=y.reset_index()
y['flperc']=y['freelunchk']/y['student']

a=y[y['ctypek']==1]['flperc']
b=y[y['ctypek']==2]['flperc']
c=y[y['ctypek']==3]['flperc']

lunch_p = stats.f_oneway(a,b,c)

y=base[['schidk', 'ctypek', 'wa', 'student']]
y=y.groupby(['schidk', 'ctypek']).sum()
y=y.reset_index()
y['waperc']=y['wa']/y['student']

a=y[y['ctypek']==1]['waperc']
b=y[y['ctypek']==2]['waperc']
c=y[y['ctypek']==3]['waperc']

wa_p = stats.f_oneway(a,b,c)

print (lunch_p, wa_p)


F_onewayResult(statistic=0.058965395387634505, pvalue=0.9427534550125046) F_onewayResult(statistic=0.02233030657687482, pvalue=0.9779192617985388)


In [56]:
df=base[base['stark']==1]

#holding list for new df at end

table1Df=[]

#List for stats for starting in kindergarten

#free lunch

flkSmall=len(df[(df['ctypek']==1) & (df['sesk']==1)])/len(df[df['ctypek']==1])
flkReg=len(df[(df['ctypek']==2) & (df['sesk']==1)])/len(df[df['ctypek']==2])
flkRegA=len(df[(df['ctypek']==3) & (df['sesk']==1)])/len(df[df['ctypek']==3])

klunch=['K', flkSmall, flkReg, flkRegA]

#white/asian

rkSmall=len(df[(df['wa']==1) & (df['ctypek']==1)])/len(df[df['ctypek']==1])
rkReg=len(df[(df['wa']==1) & (df['ctypek']==2)])/len(df[df['ctypek']==2])
rkRegA=len(df[(df['wa']==1) & (df['ctypek']==3)])/len(df[df['ctypek']==3])


krace=['K', rkSmall, rkReg, rkRegA]

#age

akSmall=df[df['ctypek']==1]['age'].mean()
akReg=df[df['ctypek']==2]['age'].mean()
akRegA=df[df['ctypek']==3]['age'].mean()



kage=['K', akSmall, akReg, akRegA]

#Attrition rate

attkSmall=len(df[(df['attritionk']==1)&(df['ctypek']==1)])/len(df[df['ctypek']==1])
attkReg=len(df[(df['attritionk']==1)&(df['ctypek']==2)])/len(df[df['ctypek']==2])
attkRegA=len(df[(df['attritionk']==1)&(df['ctypek']==3)])/len(df[df['ctypek']==3])

katt=['K', attkSmall, attkReg, attkRegA]

#class size in kindergarten

cskSmall=df[df['ctypek']==1]['csizek'].mean()
cskReg=df[df['ctypek']==2]['csizek'].mean()
cskRegA=df[df['ctypek']==3]['csizek'].mean()

kcs=['K', cskSmall, cskReg, cskRegA]

#percentile score in kindergarten

pkSmall=df[df['ctypek']==1]['score'].mean()
pkReg=df[df['ctypek']==2]['score'].mean()
pkRegA=df[df['ctypek']==3]['score'].mean()

kps=['K', pkSmall, pkReg, pkRegA]


table1Df=[klunch, krace, kage, katt, kcs, kps]

table1Df




[['K', 0.46863468634686345, 0.4762557077625571, 0.5008968609865471],
 ['K', 0.6837111228255139, 0.6753424657534246, 0.6591928251121076],
 ['K', 5.255666732788086, 5.239269256591797, 5.241255760192871],
 ['K', 0.4860305745914602, 0.5168949771689497, 0.5282511210762332],
 ['K', 15.402741432189941, 22.38127899169922, 23.20672607421875],
 ['K', 52.410121244069586, 47.422146118721464, 47.275336322869954]]

In [57]:

df=base[base['stark']==2]
df=df[df['star1']==1]
#List for stats for starting in 1st grade

#avg percentile score
df['score'].fillna(0)
df['score']=np.where(df['score']==0,(df['math1_p']+df['read1_p'])/2,df['score'])

#free lunch

fl1Small=len(df[(df['ctype1']==1) & (df['ses1']==1)])/len(df[df['ctype1']==1])
fl1Reg=len(df[(df['ctype1']==2) & (df['ses1']==1)])/len(df[df['ctype1']==2])
fl1RegA=len(df[(df['ctype1']==3) & (df['ses1']==1)])/len(df[df['ctype1']==3])

firstlunch=['First', fl1Small, fl1Reg, fl1RegA]
                     
table1Df.append(firstlunch)

#white/asian

r1Small=len(df[(df['wa']==1) & (df['ctype1']==1)])/len(df[df['ctype1']==1])
r1Reg=len(df[(df['wa']==1) & (df['ctype1']==2)])/len(df[df['ctype1']==2])
r1RegA=len(df[(df['wa']==1) & (df['ctype1']==3)])/len(df[df['ctype1']==3])

firstrace=['First', r1Small, r1Reg, r1RegA]
table1Df.append(firstrace)

#age

a1Small=df[df['ctype1']==1]['age'].mean()
a1Reg=df[df['ctype1']==2]['age'].mean()
a1RegA=df[df['ctype1']==3]['age'].mean()

firstage=['First', a1Small, a1Reg, a1RegA]
table1Df.append(firstage)

#Attrition rate

att1Small=len(df[(df['attrition1']==1)&(df['ctype1']==1)])/len(df[df['ctype1']==1])
att1Reg=len(df[(df['attrition1']==1)&(df['ctype1']==2)])/len(df[df['ctype1']==2])
att1RegA=len(df[(df['attrition1']==1)&(df['ctype1']==3)])/len(df[df['ctype1']==3])

firstatt=['First', att1Small, att1Reg, att1RegA]
table1Df.append(firstatt)

#class size in first

cs1Small=df[df['ctype1']==1]['csize1'].mean()
cs1Reg=df[df['ctype1']==2]['csize1'].mean()
cs1RegA=df[df['ctype1']==3]['csize1'].mean()

firstcs=['First', cs1Small, cs1Reg, cs1RegA]
                     
table1Df.append(firstcs)

#percentile score in first

p1Small=df[df['ctype1']==1]['score'].mean()
p1Reg=df[df['ctype1']==2]['score'].mean()
p1RegA=df[df['ctype1']==3]['score'].mean()

firstps=['First', p1Small, p1Reg, p1RegA]


table1Df.append(firstps)
                     
table1Df

[['K', 0.46863468634686345, 0.4762557077625571, 0.5008968609865471],
 ['K', 0.6837111228255139, 0.6753424657534246, 0.6591928251121076],
 ['K', 5.255666732788086, 5.239269256591797, 5.241255760192871],
 ['K', 0.4860305745914602, 0.5168949771689497, 0.5282511210762332],
 ['K', 15.402741432189941, 22.38127899169922, 23.20672607421875],
 ['K', 52.410121244069586, 47.422146118721464, 47.275336322869954],
 ['First', 0.5740740740740741, 0.6047430830039525, 0.5846501128668171],
 ['First', 0.6164021164021164, 0.5602766798418972, 0.6489841986455982],
 ['First', 5.608465671539307, 5.692687511444092, 5.691873550415039],
 ['First', 0.5211640211640212, 0.5069169960474308, 0.4582392776523702],
 ['First', 15.865078926086426, 22.686758041381836, 23.406320571899414],
 ['First', 45.71031746031746, 40.77865612648221, 44.89785553047404]]

In [58]:
#List for stats for starting in 2nd grade
df=base[base['stark']==2]
df=df[df['star1']==2]
df=df[df['star2']==1]
#avg percentile score
df['score'].fillna(0)
df['score']=np.where(df['score']==0,(df['math2_p']+df['read2_p'])/2,df['score'])

#free lunch

fl2Small=len(df[(df['ctype2']==1) & (df['ses2']==1)])/len(df[df['ctype2']==1])
fl2Reg=len(df[(df['ctype2']==2) & (df['ses2']==1)])/len(df[df['ctype2']==2])
fl2RegA=len(df[(df['ctype2']==3) & (df['ses2']==1)])/len(df[df['ctype2']==3])

x=df[(df['ctype2']==1) & (df['ses2']==1)]['score']
y=df[(df['ctype2']==2) & (df['ses2']==1)]['score']
z=df[(df['ctype2']==3) & (df['ses2']==1)]['score']

secondlunch=['Second', fl2Small, fl2Reg, fl2RegA]
                     
table1Df.append(secondlunch)

#white/asian

r2Small=len(df[(df['wa']==1) & (df['ctype2']==1)])/len(df[df['ctype2']==1])
r2Reg=len(df[(df['wa']==1) & (df['ctype2']==2)])/len(df[df['ctype2']==2])
r2RegA=len(df[(df['wa']==1) & (df['ctype2']==3)])/len(df[df['ctype2']==3])

secondrace=['Second', r2Small, r2Reg, r2RegA]
table1Df.append(secondrace)

#age

a2Small=df[df['ctype2']==1]['age'].mean()
a2Reg=df[df['ctype2']==2]['age'].mean()
a2RegA=df[df['ctype2']==3]['age'].mean()

secondage=['Second', a2Small, a2Reg, a2RegA]
table1Df.append(secondage)

#Attrition rate

att2Small=len(df[(df['attrition2']==1)&(df['ctype2']==1)])/len(df[df['ctype2']==1])
att2Reg=len(df[(df['attrition2']==1)&(df['ctype2']==2)])/len(df[df['ctype2']==2])
att2RegA=len(df[(df['attrition2']==1)&(df['ctype2']==3)])/len(df[df['ctype2']==3])

secondatt=['Second', att1Small, att1Reg, att1RegA]
table1Df.append(secondatt)

#class size in second

cs2Small=df[df['ctype2']==1]['csize2'].mean()
cs2Reg=df[df['ctype2']==2]['csize2'].mean()
cs2RegA=df[df['ctype2']==3]['csize2'].mean()

secondcs=['Second', cs1Small, cs1Reg, cs1RegA]
                     
table1Df.append(secondcs)

#percentile score in second

p2Small=df[df['ctype2']==1]['score'].mean()
p2Reg=df[df['ctype2']==2]['score'].mean()
p2RegA=df[df['ctype2']==3]['score'].mean()

secondps=['Second', p2Small, p2Reg, p2RegA]


table1Df.append(secondps)
                     
table1Df

[['K', 0.46863468634686345, 0.4762557077625571, 0.5008968609865471],
 ['K', 0.6837111228255139, 0.6753424657534246, 0.6591928251121076],
 ['K', 5.255666732788086, 5.239269256591797, 5.241255760192871],
 ['K', 0.4860305745914602, 0.5168949771689497, 0.5282511210762332],
 ['K', 15.402741432189941, 22.38127899169922, 23.20672607421875],
 ['K', 52.410121244069586, 47.422146118721464, 47.275336322869954],
 ['First', 0.5740740740740741, 0.6047430830039525, 0.5846501128668171],
 ['First', 0.6164021164021164, 0.5602766798418972, 0.6489841986455982],
 ['First', 5.608465671539307, 5.692687511444092, 5.691873550415039],
 ['First', 0.5211640211640212, 0.5069169960474308, 0.4582392776523702],
 ['First', 15.865078926086426, 22.686758041381836, 23.406320571899414],
 ['First', 45.71031746031746, 40.77865612648221, 44.89785553047404],
 ['Second', 0.6373626373626373, 0.5956454121306376, 0.608359133126935],
 ['Second', 0.532967032967033, 0.5474339035769828, 0.4504643962848297],
 ['Second', 5.725274562835

In [59]:
#List for stats for starting in 2nd grade
df=base[base['stark']==2]
df=df[df['star1']==2]
df=df[df['star2']==2]
df=df[df['star3']==1]
#avg percentile score
df['score'].fillna(0)
df['score']=np.where(df['score']==0,(df['math2_p']+df['read2_p'])/2,df['score'])

#free lunch

fl2Small=len(df[(df['ctype3']==1) & (df['ses3']==1)])/len(df[df['ctype3']==1])
fl2Reg=len(df[(df['ctype3']==2) & (df['ses3']==1)])/len(df[df['ctype3']==2])
fl2RegA=len(df[(df['ctype3']==3) & (df['ses3']==1)])/len(df[df['ctype3']==3])

secondlunch=['Third', fl2Small, fl2Reg, fl2RegA]
                     
table1Df.append(secondlunch)

#white/asian

r2Small=len(df[(df['wa']==1) & (df['ctype3']==1)])/len(df[df['ctype3']==1])
r2Reg=len(df[(df['wa']==1) & (df['ctype3']==2)])/len(df[df['ctype3']==2])
r2RegA=len(df[(df['wa']==1) & (df['ctype3']==3)])/len(df[df['ctype3']==3])

secondrace=['Third', r2Small, r2Reg, r2RegA]
table1Df.append(secondrace)

#age

a2Small=df[df['ctype3']==1]['age'].mean()
a2Reg=df[df['ctype3']==2]['age'].mean()
a2RegA=df[df['ctype3']==3]['age'].mean()

secondage=['Third', a2Small, a2Reg, a2RegA]
table1Df.append(secondage)

#Attrition rate

#NA for year three

secondatt=['Third', 'NA', 'NA', 'NA']
table1Df.append(secondatt)

#class size in second

cs2Small=df[df['ctype3']==1]['csize3'].mean()
cs2Reg=df[df['ctype3']==2]['csize3'].mean()
cs2RegA=df[df['ctype3']==3]['csize3'].mean()

secondcs=['Third', cs1Small, cs1Reg, cs1RegA]
                     
table1Df.append(secondcs)

#percentile score in second

p2Small=df[df['ctype3']==1]['score'].mean()
p2Reg=df[df['ctype3']==2]['score'].mean()
p2RegA=df[df['ctype3']==3]['score'].mean()

secondps=['Third', p2Small, p2Reg, p2RegA]


table1Df.append(secondps)

In [60]:
headers=['Grade', 'Small', 'Regular', 'Regular/Aide']
tableOne=pd.DataFrame(data=table1Df, columns=headers)

tableOne.to_csv('table1.csv')

tableOne

,Grade,Small,Regular,Regular/Aide
0,K,0.468635,0.476256,0.500897
1,K,0.683711,0.675342,0.659193
2,K,5.25567,5.23927,5.24126
3,K,0.486031,0.516895,0.528251
4,K,15.4027,22.3813,23.2067
5,K,52.4101,47.4221,47.2753
6,First,0.574074,0.604743,0.58465
7,First,0.616402,0.560277,0.648984
8,First,5.60847,5.69269,5.69187
9,First,0.521164,0.506917,0.458239


In [61]:
# Table 2:  OLS Kindergarten

df=base[base['stark']==1]

df=df[['kscore','smallc', 'regAc', 'reg', 'wa', 'sex','freelunchk', 'csizek']]

df['beta3']=df['wa']*df['sex']*df['freelunchk']
df=df[df['kscore']>0]
X=df[['smallc', 'regAc']]
y=df['kscore']

mod=sm.OLS(y, sm.add_constant(X))
res=mod.fit()
#print (res.summary())

res_robust = res.get_robustcov_results()

print (res_robust.summary())

#print(res_robust.summary().as_latex())
#print (len(df))'''

                            OLS Regression Results                            
Dep. Variable:                 kscore   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     19.95
Date:                Tue, 26 Mar 2019   Prob (F-statistic):           2.32e-09
Time:                        20:57:28   Log-Likelihood:                -27704.
No. Observations:                5870   AIC:                         5.541e+04
Df Residuals:                    5867   BIC:                         5.543e+04
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         48.8577      0.611     79.976      0.0

In [62]:
# Table 2:  OLS Kindergarten with fixed effects

df=base[base['stark']==1]

df=df[['kscore','smallc', 'regAc', 'reg', 'wa', 'sex','freelunchk', 'csizek', 'schidk','student']]

df['beta3']=df['wa']*df['sex']*df['freelunchk']
df=df[df['kscore']>0]
X=df[['smallc', 'regAc']]
y=df['kscore']

# with fixed effects

df=df.set_index(['schidk', 'student'])
mod=p(df['kscore'], df[['smallc','regAc']], entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print (res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 kscore   R-squared:                        0.0110
Estimator:                   PanelOLS   R-squared (Between):              0.0588
No. Observations:                5870   R-squared (Within):               0.0000
Date:                Tue, Mar 26 2019   R-squared (Overall):              0.0506
Time:                        20:57:28   Log-likelihood                -2.689e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      32.075
Entities:                          79   P-value                           0.0000
Avg Obs:                       74.304   Distribution:                  F(2,5789)
Min Obs:                       34.000                                           
Max Obs:                       138.00   F-statistic (robust):             9.6717
                            

In [63]:
# Table 2:  OLS Kindergarten with fixed effects

df=base[base['stark']==1]

df=df[['kscore','smallc', 'regAc', 'reg', 'wa', 'sex','freelunchk', 'csizek', 'schidk','student']]

df['beta3']=df['wa']*df['sex']*df['freelunchk']
df=df[df['kscore']>0]

# with fixed effects

df=df.set_index(['schidk', 'student'])
mod=p(df['kscore'], df[['smallc','regAc', 'wa', 'sex', 'freelunchk']], entity_effects=True)
res = mod.fit(cov_type='clustered', cluster_entity=True)
print (res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                 kscore   R-squared:                        0.0906
Estimator:                   PanelOLS   R-squared (Between):              0.1693
No. Observations:                5870   R-squared (Within):               0.0000
Date:                Tue, Mar 26 2019   R-squared (Overall):              0.1460
Time:                        20:57:28   Log-likelihood                -2.665e+04
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      115.33
Entities:                          79   P-value                           0.0000
Avg Obs:                       74.304   Distribution:                  F(5,5786)
Min Obs:                       34.000                                           
Max Obs:                       138.00   F-statistic (robust):             63.626
                            

In [64]:
res

Dep. Variable:,kscore,R-squared:,0.0906
Estimator:,PanelOLS,R-squared (Between):,0.1693
No. Observations:,5870,R-squared (Within):,0.0000
Date:,"Tue, Mar 26 2019",R-squared (Overall):,0.1460
Time:,20:57:28,Log-likelihood,-2.665e+04
Cov. Estimator:,Clustered,,
,,F-statistic:,115.33
Entities:,79,P-value,0.0000
Avg Obs:,74.304,Distribution:,"F(5,5786)"
Min Obs:,34.000,,
Max Obs:,138.00,F-statistic (robust):,63.626


In [65]:
df=base
df.head()

,id,sex,race,mathk,readk,math1,read1,math2,read2,math3,...,attrition2,wa,age,score,score1,smallc,regAc,reg,student,kscore
0,38495.0,0,2,0.0,0.0,0.0,0.0,0.0,0.0,510.0,...,1,0,6.0,1.0,1.0,0,0,0,1,0.0
1,114890.0,0,2,0.0,0.0,0.0,0.0,0.0,0.0,536.0,...,1,0,7.0,1.5,1.5,0,0,0,1,0.0
2,32145.0,0,2,0.0,0.0,0.0,0.0,0.0,0.0,571.0,...,0,0,5.0,7.0,7.0,0,0,0,1,0.0
3,121064.0,1,2,0.0,0.0,515.0,456.0,545.0,571.0,554.0,...,0,0,6.0,25.5,25.5,0,0,0,1,0.0
4,116042.0,0,1,0.0,0.0,0.0,0.0,0.0,0.0,515.0,...,1,1,7.0,1.0,1.0,0,0,0,1,0.0


In [149]:
#Question 4

df=base
df['init']=np.where(df['ctypek']==0, df['ctype1'], df['ctypek'])
df['init']=np.where(df['init']==0, df['ctype2'], df['init'])
df['init']=np.where(df['init']==0, df['ctype3'], df['init'])

df=df[df['init']!=0]
len(df)

11505

In [150]:
# predict class size using assignment

# Table 2:  OLS Kindergarten

df=df[['csizek','init','score', 'ctypek', 'kscore', 'score1','csize1','score2', 'csize2', 'csize3', 'score3','ctype3','ctype2']]

df['dummy1']=np.where(df['init']==1,1,0)
df['dummy2']=np.where(df['init']!=1,1,0)

X=df[['dummy1', 'dummy2']]
y=df['csizek']

mod=sm.OLS(y, sm.add_constant(X))
res=mod.fit()
print (res.summary())

#res_robust = res.get_robustcov_results()

#print (res_robust.summary())

#print(res.summary().as_latex())
#print (len(df))'''

                            OLS Regression Results                            
Dep. Variable:                 csizek   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     90.39
Date:                Tue, 26 Mar 2019   Prob (F-statistic):           2.34e-21
Time:                        21:57:34   Log-Likelihood:                -43627.
No. Observations:               11505   AIC:                         8.726e+04
Df Residuals:                   11503   BIC:                         8.727e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          7.1885      0.076     94.759      0.0

In [176]:
#Kindergarten two stage

df['const']=1
X=df[['const','init']]
y=df['ctypek']

results_fs = sm.OLS(y,
                    X,
                    missing='drop').fit()
df['predicted_type'] = results_fs.predict()


X2=df[['const', 'predicted_type']]
y=df['score']
results_2s = sm.OLS(y,
                    X2,
                    missing='drop').fit()

print(results_2s.summary())



                            OLS Regression Results                            
Dep. Variable:                  score   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     55.98
Date:                Tue, 26 Mar 2019   Prob (F-statistic):           8.38e-14
Time:                        22:28:33   Log-Likelihood:                -27871.
No. Observations:                6026   AIC:                         5.575e+04
Df Residuals:                    6024   BIC:                         5.576e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             61.1256      1.050     58.

In [168]:
#first grade IV OLS


df['const']=1
X=df[['const','init']]
y=df['csize1']

results_fs = sm.OLS(y,
                    X,
                    missing='drop').fit()
df['predicted_size'] = results_fs.predict()


X2=df[['const', 'predicted_size', 'dummy1', 'dummy2']]
y=df['score1']
results_2s = sm.OLS(y,
                    X2,
                    missing='drop').fit()

print(results_2s.summary())


                            OLS Regression Results                            
Dep. Variable:                 score1   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     31.62
Date:                Tue, 26 Mar 2019   Prob (F-statistic):           2.19e-14
Time:                        22:15:33   Log-Likelihood:                -27868.
No. Observations:                6026   AIC:                         5.574e+04
Df Residuals:                    6023   BIC:                         5.576e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             41.6710      3.628     11.

In [169]:
#2nd grade OLS

df['const']=1
X=df[['const','init']]
y=df['csize2']

results_fs = sm.OLS(y,
                    X,
                    missing='drop').fit()
df['predicted_size'] = results_fs.predict()


X2=df[['const', 'predicted_size', 'dummy1', 'dummy2']]
y=df['score2']
results_2s = sm.OLS(y,
                    X2,
                    missing='drop').fit()

print(results_2s.summary())

                            OLS Regression Results                            
Dep. Variable:                 score2   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     25.48
Date:                Tue, 26 Mar 2019   Prob (F-statistic):           9.56e-12
Time:                        22:16:45   Log-Likelihood:                -28020.
No. Observations:                6026   AIC:                         5.605e+04
Df Residuals:                    6023   BIC:                         5.607e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             39.2216      2.892     13.

In [170]:
df['const']=1
X=df[['const','init']]
y=df['csize3']

results_fs = sm.OLS(y,
                    X,
                    missing='drop').fit()
df['predicted_size'] = results_fs.predict()


X2=df[['const', 'predicted_size', 'dummy1', 'dummy2']]
y=df['score3']
results_2s = sm.OLS(y,
                    X2,
                    missing='drop').fit()

print(results_2s.summary())

                            OLS Regression Results                            
Dep. Variable:                 score3   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     27.11
Date:                Tue, 26 Mar 2019   Prob (F-statistic):           1.91e-12
Time:                        22:17:48   Log-Likelihood:                -28287.
No. Observations:                6026   AIC:                         5.658e+04
Df Residuals:                    6023   BIC:                         5.660e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             38.4195      3.091     12.